Pre-requisties

1. Mount Lakehouse to the notebook 
2. Added the dataset.parquet file location as abfss path
3. Add your eventstream connection details , i.e Eventhub name and connection string

In [ ]:
!pip install psutil asyncio azure-eventhub

StatementMeta(, 196901ef-6aa1-4fa5-bd7b-05443c94b3b7, 8, Finished, Available, Finished)

In [ ]:
import threading

StatementMeta(, 196901ef-6aa1-4fa5-bd7b-05443c94b3b7, 9, Finished, Available, Finished)

In [ ]:
df = spark.read.parquet("abfss://Hackathon@onelake.dfs.fabric.microsoft.com/hack_lakehouse.Lakehouse/Files/streaming_data.parquet")
print(df, type(df))

StatementMeta(, 196901ef-6aa1-4fa5-bd7b-05443c94b3b7, 10, Finished, Available, Finished)

DataFrame[latitude: double, longitude: double, Elevation: double, id: bigint, heart_rate: bigint, Systolic_BP: double, Diastolic_BP: double, stress: double] <class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
def create_panic_session(id, user_id, lat, long, timestamp):
    new_session_id = 1
    # Example of query for reading data from Kusto. Replace T with your <tablename>.
    kustoQuery = f"panic_logs | where user_id == {user_id} | sort by session_id desc | top 1 by session_id"
    # The query URI for reading the data e.g. https://<>.kusto.data.microsoft.com.
    kustoUri = "https://trd-mwdyc69szk2uga2n12.z0.kusto.fabric.microsoft.com"
    # The database with data to be read.
    database = "KQL_DB"
    # The access credentials.
    accessToken = mssparkutils.credentials.getToken(kustoUri)
    kustoDf = spark.read \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("accessToken", accessToken) \
        .option("kustoCluster", kustoUri) \
        .option("kustoDatabase", database) \
        .option("kustoQuery", kustoQuery).load()

    try:
        session = list(kustoDf.select("session_id").collect())[0][0]
        new_session_id = session + 1
    except IndexError:
        new_session_id = 1

    df = spark.createDataFrame(
        [(new_session_id, id, user_id, float(lat), float(long), timestamp)],
        ["session_id", "stream_id", "user_id", "panic_latitude", "panic_longitude", "timestamp"]
    )

    df.write \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("kustoCluster", kustoUri) \
        .option("kustoDatabase", database) \
        .option("kustoTable", "panic_logs") \
        .option("accessToken", accessToken) \
        .option("tableCreateOptions", "CreateIfNotExist") \
        .mode("Append").save()

    return new_session_id


StatementMeta(, 196901ef-6aa1-4fa5-bd7b-05443c94b3b7, 11, Finished, Available, Finished)

In [ ]:
import psutil
import time
import json
import asyncio
from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient
from pytz import timezone 
from datetime import datetime

EVENTSTREAM_CONNECTION_STR = "Endpoint=sb://esehpnvpd2z3rh2cz9mhzx.servicebus.windows.net/;SharedAccessKeyName=key_d87b5db7-c82d-4ce3-a4aa-2f0edcb60065;SharedAccessKey=PCbbdouQb12uQKS6VQR9lxeEIgiy+zxmY+AEhG94ovM=;EntityPath=es_f45b1791-0674-4186-bbe3-58e46cb0375d"
EVENTSTREAM_NAME = "es_f45b1791-0674-4186-bbe3-58e46cb0375d"

    # Create a producer client to send messages to the EventStream.
    # Specify a connection string to your EventStream namespace and the eventstream name.
producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENTSTREAM_CONNECTION_STR, eventhub_name=EVENTSTREAM_NAME
)
total_df_length = df.count()
count = 0

user_is_panic = False
session_updated = False
# ob = threading.Thread(target=create_panic_session, args = (id, user_id, lat, long, timestamp, ))
while True:
    event_data_batch = await producer.create_batch()
    timestamp = datetime.now(timezone("Asia/Kolkata")).strftime('%Y-%m-%d %H:%M:%S.%f')
    row = df.collect()
    if count >= 100 and count < 500 :
        user_is_panic = True
    else :
        user_is_panic = False

    user_data = {
            "id" : row[count].__getitem__("id"),
            "user_id" : 1,
            "user_latitude" : row[count].__getitem__("latitude"),
            "user_longitude" : row[count].__getitem__("longitude"),
            "user_heart_rate" : row[count].__getitem__("heart_rate"),
            "user_dia_blood_pressure" : row[count].__getitem__("Diastolic_BP"),
            "user_sys_blood_pressure" : row[count].__getitem__("Systolic_BP"),
            "user_stress" : row[count].__getitem__("stress"),
            "user_is_panic" : user_is_panic,
            "timeStamp" : f'{datetime.now()}'
    }

            
    if user_is_panic:
        if not session_updated :
            session_id = create_panic_session(
                    user_data["id"], 
                    user_data["user_id"],
                    user_data["user_latitude"],
                    user_data["user_longitude"],
                    user_data["timeStamp"]
        )
        session_updated = True
        user_data["panic_session_id"] = session_id
        
    else :
        user_data["panic_session_id"] = -1

    json_data = json.dumps(user_data)
    print(count,json_data)
    event_data_batch.add(EventData(json_data))
    # Send the batch of events to the event hub.
    await producer.send_event(event_data_batch)
    time.sleep(1)
    count += 1
    if count == total_df_length:
            break

StatementMeta(, 196901ef-6aa1-4fa5-bd7b-05443c94b3b7, 12, Submitted, Running, Running)

0 {"id": 0, "user_id": 1, "user_latitude": 18.39751, "user_longitude": 73.85768, "user_heart_rate": 93, "user_dia_blood_pressure": 83.89597865975807, "user_sys_blood_pressure": 123.13475246264036, "user_stress": null, "user_is_panic": false, "timeStamp": "2024-11-12 18:51:23.214479", "panic_session_id": -1}
1 {"id": 1, "user_id": 1, "user_latitude": 18.39751, "user_longitude": 73.8577, "user_heart_rate": 91, "user_dia_blood_pressure": 71.86059947355578, "user_sys_blood_pressure": 125.31795505412154, "user_stress": 14.179369018078773, "user_is_panic": false, "timeStamp": "2024-11-12 18:51:24.322154", "panic_session_id": -1}
2 {"id": 2, "user_id": 1, "user_latitude": 18.39749, "user_longitude": 73.85774, "user_heart_rate": 96, "user_dia_blood_pressure": 78.6253847942726, "user_sys_blood_pressure": 129.9532805894792, "user_stress": 26.271042896143932, "user_is_panic": false, "timeStamp": "2024-11-12 18:51:25.451795", "panic_session_id": -1}
3 {"id": 3, "user_id": 1, "user_latitude": 18.39

In [ ]:
def create_panic_session(id, user_id, lat, long, timestamp):
    new_session_id = 1
    # Example of query for reading data from Kusto. Replace T with your <tablename>.
    kustoQuery = f"panic_logs | where user_id == {user_id} | sort by session_id desc | top 1 by session_id"
    # The query URI for reading the data e.g. https://<>.kusto.data.microsoft.com.
    kustoUri = "https://trd-mwdyc69szk2uga2n12.z0.kusto.fabric.microsoft.com"
    # The database with data to be read.
    database = "KQL_DB"
    # The access credentials.
    accessToken = mssparkutils.credentials.getToken(kustoUri)
    kustoDf = spark.read \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("accessToken", accessToken) \
        .option("kustoCluster", kustoUri) \
        .option("kustoDatabase", database) \
        .option("kustoQuery", kustoQuery).load()

    try:
        session = list(kustoDf.select("session_id").collect())[0][0]
        new_session_id = session + 1
    except IndexError:
        new_session_id = 1

    df = spark.createDataFrame(
        [(new_session_id, id, user_id, float(lat), float(long), timestamp)],
        ["session_id", "stream_id", "user_id", "panic_latitude", "panic_longitude", "timestamp"]
    )

    df.write \
        .format("com.microsoft.kusto.spark.synapse.datasource") \
        .option("kustoCluster", kustoUri) \
        .option("kustoDatabase", database) \
        .option("kustoTable", "panic_logs") \
        .option("accessToken", accessToken) \
        .option("tableCreateOptions", "CreateIfNotExist") \
        .mode("Append").save()

    return new_session_id

# Example usage
create_panic_session(0, 0, 1, 1, 1)
